# **Loading Dataset**

In [4]:
!git clone https://github.com/dml-qom/FarsTail.git
!ls FarsTail/data

import pandas as pd

test = pd.read_csv('FarsTail/data/Test-word.csv', sep='\t')
train = pd.read_csv('FarsTail/data/Train-word.csv', sep='\t')
val = pd.read_csv('FarsTail/data/Val-word.csv', sep='\t')


Cloning into 'FarsTail'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 137 (delta 26), reused 18 (delta 18), pack-reused 104 (from 1)
Receiving objects: 100% (137/137), 43.86 MiB | 11.82 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Indexed-FarsTail.npz  Test-word.csv  Train-word.csv  Val-word.csv


In [5]:
val.head()

,premise,hypothesis,label
0,یکی از سرآمدانی که بر تارک علمی مدرسه اسکندریه...,کتاب مگیستی (المجسطی) از آثار بطلمیوس است و از...,e
1,یکی از سرآمدانی که بر تارک علمی مدرسه اسکندریه...,المجسطی (مگیستی)، یکی از کتاب های هرون از مکتب...,c
2,یکی از سرآمدانی که بر تارک علمی مدرسه اسکندریه...,ترجمه المجسطی، پایه تهیه جداول نجومی گردید که ...,n
3,تاریخ پزشکی یونانی با بقراط معروف است. آثار طب...,کتاب کلمات قصار یا فصول، اثر بقراط است.,e
4,تاریخ پزشکی یونانی با بقراط معروف است. آثار طب...,جالینوس، نویسنده کتاب کلمات قصار بوده است.,c


# **Precision Recall f1**

In [ ]:
def precision_recall_f1(y_true, y_pred, labels=['e', 'c', 'n']):
    results = {}
    for label in labels:
        tp = sum((y_true[i]==label and y_pred[i]==label) for i in range(len(y_true)))
        fp = sum((y_true[i]!=label and y_pred[i]==label) for i in range(len(y_true)))
        fn = sum((y_true[i]==label and y_pred[i]!=label) for i in range(len(y_true)))

        precision = tp / (tp + fp + 1e-10)
        recall = tp / (tp + fn + 1e-10)
        f1 = 2 * precision * recall / (precision + recall + 1e-10)

        results[label] = {'precision': precision, 'recall': recall, 'f1': f1}

    # macro
    macro_f1 = sum(v['f1'] for v in results.values()) / len(labels)
    results['macro_f1'] = macro_f1
    return results


# **login huggingface using access token**

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "google/gemma-3-270m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")

def generate_response(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [9]:
zero_prompts = [
    # Prompt 1
    "با توجه به جملات زیر، مشخص کن که جمله دوم با جمله اول چه رابطه‌ای دارد: \n"
    "گزینه‌ها: e=نتیجه‌گیری، c=تضاد، n=بی‌ربط.\n"
    "فرض: {premise}\n"
    "فرضیه: {hypothesis}\n"
    "پاسخ فقط یکی از e، c یا n باشد.",

    # Prompt 2
    "تحلیل رابطه بین دو جمله فارسی زیر را انجام بده و یکی از برچسب‌های e، c یا n را انتخاب کن:\n"
    "۱) {premise}\n۲) {hypothesis}\n"
    "e یعنی جمله دوم نتیجه اول است، c یعنی متناقض است، n یعنی بی‌ربط است.\nپاسخ:",

    # Prompt 3
    "جمله اول را پایه قرار بده و بگو آیا جمله دوم از آن نتیجه می‌شود (e)، با آن تضاد دارد (c)، یا هیچ رابطه‌ای ندارد (n):\n"
    "جمله اول: {premise}\nجمله دوم: {hypothesis}\nنتیجه:",

    # Prompt 4
    "بین دو جمله‌ی فارسی زیر چه رابطه‌ای برقرار است؟ فقط یکی از e/c/n بنویس.\n"
    "جمله اول: {premise}\nجمله دوم: {hypothesis}\nپاسخ:",

    # Prompt 5
    "درک معنایی: آیا جمله دوم از جمله اول نتیجه می‌شود (e)، با آن در تضاد است (c)، یا مستقل است (n)؟\n"
    "فرض: {premise}\nفرضیه: {hypothesis}\nپاسخ:"
]


In [10]:
few_prompts = [
    """
 با توجه به جملات زیر، مشخص کن که جمله دوم با جمله اول چه رابطه‌ای دارد:
گزینه‌ها: e=نتیجه‌گیری، c=تضاد، n=بی‌ربط.
پاسخ فقط یکی از e، c یا n باشد
    فرض: رواج و پیدایش علم کلام، از مهمترین نتایج ورود اندیشه ها به جهان اسلام و تلاش و مساعی معتزلیان و مناظرات آنان با مخالفان بود.
    فرضیه: یکی از مهمترین نتایج ورود اندیشه‌ها به عالم اسلام و سعی و تلاش معتزلیان، رواج علم کلام است.
    پاسخ: e

    فرض: زادروز پروین اعتصامی (بیست و پنجم اسفندماه)، به عنوان «روز بزرگداشت پروین اعتصامی» نام‌گذاری شده‌است.
    فرضیه: ۲۵ اسفند در تقویم ایرانی، روز بزرگداشت پروین اعتصامی می باشد.
    پاسخ: e

    فرض: در دوران نبوت، مسجد النبی اولین آموزشگاه آن دوره بود.
    فرضیه: اولین آموزشگاه عصر نبوت، مسجد کوفه نام دارد.
    پاسخ: c

    فرض: ویژگی های هنر عصر اموی: ۱- تلفیقی بودن ۲- بازنمایی نوعی تفنن و تفریح ۳- نقاشی های تزئینی و تندیس های بی کیفیت
    فرضیه: با کیفیت بودن تندیس های دوره اموی، یکی از ویژگی های هنر این دوره است.
    پاسخ: c

    فرض: محاصره اقتصادی پیامبر (ص) و یارانش که در سال هفتم بعثت شروع شد، سه سال طول کشید و یکی از سخت‌ترین دوره‌های زندگی پیامبر و مسلمانان به شمار می‌رود. اسلام هرگونه هوا پرستی، مقام‏ پرستی، پول‏ پرستی، شخص ‏پرستی را شرک می‌شمارد.
    فرضیه: در دوران محاصره اقتصادی رسول اکرم (ص) و پیروانش، کسی حاضر به معامله با آنان نبود.
    پاسخ: n

    فرض: امام صادق (ع) در رابطه با مرگ می‌فرمایند: «مرگ برای اهل ایمان شبیه بیرون آوردن گوهری از صندوق یا پریدن مرغ از قفس است.»
    فرضیه: انسان‌هایی که از دید امام صادق (ع)، مرگ برایشان مانند پریدن مرغ از قفس است، هدف غایی خود را در دنیای زودگذر قرار نداده‌اند.
    پاسخ: n
    ---
    حالا:
    فرض: {premise}
    فرضیه: {hypothesis}
    پاسخ:""",

    """با توجه به مثال‌ها رابطه را مشخص کن:
    مثال۱:
    فرض: رواج و پیدایش علم کلام، از مهمترین نتایج ورود اندیشه ها به جهان اسلام و تلاش و مساعی معتزلیان و مناظرات آنان با مخالفان بود.
    فرضیه: یکی از مهمترین نتایج ورود اندیشه‌ها به عالم اسلام و سعی و تلاش معتزلیان، رواج علم کلام است.
    → e

    مثال۲:
    فرض: زادروز پروین اعتصامی (بیست و پنجم اسفندماه)، به عنوان «روز بزرگداشت پروین اعتصامی» نام‌گذاری شده‌است.
    فرضیه: ۲۵ اسفند در تقویم ایرانی، روز بزرگداشت پروین اعتصامی می باشد.
    → e

    مثال۳:
    فرض: در دوران نبوت، مسجد النبی اولین آموزشگاه آن دوره بود.
    فرضیه: اولین آموزشگاه عصر نبوت، مسجد کوفه نام دارد.
    → c

    مثال۴:
    فرض: ویژگی های هنر عصر اموی: ۱- تلفیقی بودن ۲- بازنمایی نوعی تفنن و تفریح ۳- نقاشی های تزئینی و تندیس های بی کیفیت
    فرضیه: با کیفیت بودن تندیس های دوره اموی، یکی از ویژگی های هنر این دوره است.
    → c

    مثال۵:
    فرض: محاصره اقتصادی پیامبر (ص) و یارانش که در سال هفتم بعثت شروع شد، سه سال طول کشید و یکی از سخت‌ترین دوره‌های زندگی پیامبر و مسلمانان به شمار می‌رود. اسلام هرگونه هوا پرستی، مقام‏ پرستی، پول‏ پرستی، شخص ‏پرستی را شرک می‌شمارد.
    فرضیه: در دوران محاصره اقتصادی رسول اکرم (ص) و پیروانش، کسی حاضر به معامله با آنان نبود.
    → n

    مثال۶:
    فرض: امام صادق (ع) در رابطه با مرگ می‌فرمایند: «مرگ برای اهل ایمان شبیه بیرون آوردن گوهری از صندوق یا پریدن مرغ از قفس است.»
    فرضیه: انسان‌هایی که از دید امام صادق (ع)، مرگ برایشان مانند پریدن مرغ از قفس است، هدف غایی خود را در دنیای زودگذر قرار نداده‌اند.
    → n
    ---
    اکنون:
    فرض: {premise}
    فرضیه: {hypothesis}
    پاسخ:""",

    """چند نمونه‌ی آموزشی:
    ۱. فرض: رواج و پیدایش علم کلام، از مهمترین نتایج ورود اندیشه ها به جهان اسلام و تلاش و مساعی معتزلیان و مناظرات آنان با مخالفان بود.
       فرضیه: یکی از مهمترین نتایج ورود اندیشه‌ها به عالم اسلام و سعی و تلاش معتزلیان، رواج علم کلام است.
       → e

    ۲. فرض: زادروز پروین اعتصامی (بیست و پنجم اسفندماه)، به عنوان «روز بزرگداشت پروین اعتصامی» نام‌گذاری شده‌است.
       فرضیه: ۲۵ اسفند در تقویم ایرانی، روز بزرگداشت پروین اعتصامی می باشد.
       → e

    ۳. فرض: در دوران نبوت، مسجد النبی اولین آموزشگاه آن دوره بود.
       فرضیه: اولین آموزشگاه عصر نبوت، مسجد کوفه نام دارد.
       → c

    ۴. فرض: ویژگی های هنر عصر اموی: ۱- تلفیقی بودن ۲- بازنمایی نوعی تفنن و تفریح ۳- نقاشی های تزئینی و تندیس های بی کیفیت
       فرضیه: با کیفیت بودن تندیس های دوره اموی، یکی از ویژگی های هنر این دوره است.
       → c

    ۵. فرض: محاصره اقتصادی پیامبر (ص) و یارانش که در سال هفتم بعثت شروع شد، سه سال طول کشید و یکی از سخت‌ترین دوره‌های زندگی پیامبر و مسلمانان به شمار می‌رود. اسلام هرگونه هوا پرستی، مقام‏ پرستی، پول‏ پرستی، شخص ‏پرستی را شرک می‌شمارد.
       فرضیه: در دوران محاصره اقتصادی رسول اکرم (ص) و پیروانش، کسی حاضر به معامله با آنان نبود.
       → n

    ۶. فرض: امام صادق (ع) در رابطه با مرگ می‌فرمایند: «مرگ برای اهل ایمان شبیه بیرون آوردن گوهری از صندوق یا پریدن مرغ از قفس است.»
       فرضیه: انسان‌هایی که از دید امام صادق (ع)، مرگ برایشان مانند پریدن مرغ از قفس است، هدف غایی خود را در دنیای زودگذر قرار نداده‌اند.
       → n
    ---
    حالا برای داده‌ی زیر پاسخ بده:
    فرض: {premise}
    فرضیه: {hypothesis}
    نتیجه:""",

    """در مثال‌های زیر، نوع رابطه بین دو جمله مشخص شده است:
    مثال۱: (e)
    فرض: رواج و پیدایش علم کلام، از مهمترین نتایج ورود اندیشه ها به جهان اسلام و تلاش و مساعی معتزلیان و مناظرات آنان با مخالفان بود.
    فرضیه: یکی از مهمترین نتایج ورود اندیشه‌ها به عالم اسلام و سعی و تلاش معتزلیان، رواج علم کلام است.

    مثال۲: (e)
    فرض: زادروز پروین اعتصامی (بیست و پنجم اسفندماه)، به عنوان «روز بزرگداشت پروین اعتصامی» نام‌گذاری شده‌است.
    فرضیه: ۲۵ اسفند در تقویم ایرانی، روز بزرگداشت پروین اعتصامی می باشد.

    مثال۳: (c)
    فرض: در دوران نبوت، مسجد النبی اولین آموزشگاه آن دوره بود.
    فرضیه: اولین آموزشگاه عصر نبوت، مسجد کوفه نام دارد.

    مثال۴: (c)
    فرض: ویژگی های هنر عصر اموی: ۱- تلفیقی بودن ۲- بازنمایی نوعی تفنن و تفریح ۳- نقاشی های تزئینی و تندیس های بی کیفیت
    فرضیه: با کیفیت بودن تندیس های دوره اموی، یکی از ویژگی های هنر این دوره است.

    مثال۵: (n)
    فرض: محاصره اقتصادی پیامبر (ص) و یارانش که در سال هفتم بعثت شروع شد، سه سال طول کشید و یکی از سخت‌ترین دوره‌های زندگی پیامبر و مسلمانان به شمار می‌رود. اسلام هرگونه هوا پرستی، مقام‏ پرستی، پول‏ پرستی، شخص ‏پرستی را شرک می‌شمارد.
    فرضیه: در دوران محاصره اقتصادی رسول اکرم (ص) و پیروانش، کسی حاضر به معامله با آنان نبود.

    مثال۶: (n)
    فرض: امام صادق (ع) در رابطه با مرگ می‌فرمایند: «مرگ برای اهل ایمان شبیه بیرون آوردن گوهری از صندوق یا پریدن مرغ از قفس است.»
    فرضیه: انسان‌هایی که از دید امام صادق (ع)، مرگ برایشان مانند پریدن مرغ از قفس است، هدف غایی خود را در دنیای زودگذر قرار نداده‌اند.
    ---
    داده جدید:
    {premise}
    {hypothesis}
    پاسخ:""",

    """بیایید رابطه‌ی معنایی را بیابیم. از نمونه‌ها یاد بگیر:
    فرض: رواج و پیدایش علم کلام، از مهمترین نتایج ورود اندیشه ها به جهان اسلام و تلاش و مساعی معتزلیان و مناظرات آنان با مخالفان بود.
    فرضیه: یکی از مهمترین نتایج ورود اندیشه‌ها به عالم اسلام و سعی و تلاش معتزلیان، رواج علم کلام است.
    → e

    فرض: زادروز پروین اعتصامی (بیست و پنجم اسفندماه)، به عنوان «روز بزرگداشت پروین اعتصامی» نام‌گذاری شده‌است.
    فرضیه: ۲۵ اسفند در تقویم ایرانی، روز بزرگداشت پروین اعتصامی می باشد.
    → e

    فرض: در دوران نبوت، مسجد النبی اولین آموزشگاه آن دوره بود.
    فرضیه: اولین آموزشگاه عصر نبوت، مسجد کوفه نام دارد.
    → c

    فرض: ویژگی های هنر عصر اموی: ۱- تلفیقی بودن ۲- بازنمایی نوعی تفنن و تفریح ۳- نقاشی های تزئینی و تندیس های بی کیفیت
    فرضیه: با کیفیت بودن تندیس های دوره اموی، یکی از ویژگی های هنر این دوره است.
    → c

    فرض: محاصره اقتصادی پیامبر (ص) و یارانش که در سال هفتم بعثت شروع شد، سه سال طول کشید و یکی از سخت‌ترین دوره‌های زندگی پیامبر و مسلمانان به شمار می‌رود. اسلام هرگونه هوا پرستی، مقام‏ پرستی، پول‏ پرستی، شخص ‏پرستی را شرک می‌شمارد.
    فرضیه: در دوران محاصره اقتصادی رسول اکرم (ص) و پیروانش، کسی حاضر به معامله با آنان نبود.
    → n

    فرض: امام صادق (ع) در رابطه با مرگ می‌فرمایند: «مرگ برای اهل ایمان شبیه بیرون آوردن گوهری از صندوق یا پریدن مرغ از قفس است.»
    فرضیه: انسان‌هایی که از دید امام صادق (ع)، مرگ برایشان مانند پریدن مرغ از قفس است، هدف غایی خود را در دنیای زودگذر قرار نداده‌اند.
    → n
    ---
    حالا:
    فرض: {premise}
    فرضیه: {hypothesis}
    پاسخ:"""
]


In [11]:
import random

def evaluate_prompt(prompt_template, data, n=5 ):
    samples = data.sample(n)
    y_true, y_pred = [], []
    for _, row in samples.iterrows():
        p = prompt_template.format(premise=row['premise'], hypothesis=row['hypothesis'])
        response = generate_response(p)
        pred = 'e' if 'e' in response else ('c' if 'c' in response else 'n')
        y_true.append(row['label'])
        y_pred.append(pred)
    return precision_recall_f1(y_true, y_pred)


In [12]:
results_zero = {f"zero_{i+1}": evaluate_prompt(p, val) for i,p in enumerate(zero_prompts)}
results_few = {f"few_{i+1}": evaluate_prompt(p, val) for i,p in enumerate(few_prompts)}

print("Zero-shot results:")
for k,v in results_zero.items():
    print(k, v['macro_f1'])

print("Few-shot results:")
for k,v in results_few.items():
    print(k, v['macro_f1'])


Zero-shot results:
zero_1 0.11111111109814814
zero_2 0.0
zero_3 0.0
zero_4 0.19047619045714284
zero_5 0.19047619045714284
Few-shot results:
few_1 0.11111111109814814
few_2 0.24999999997812503
few_3 0.11111111109814814
few_4 0.19047619045714284
few_5 0.11111111109814814


In [13]:
best_zero = sorted(results_zero.items(), key=lambda x: x[1]['macro_f1'], reverse=True)[:3]
best_few = sorted(results_few.items(), key=lambda x: x[1]['macro_f1'], reverse=True)[:3]

print("Best 3 Zero-shot Prompts:")
for name,_ in best_zero:
    print(name, "\n", zero_prompts[int(name.split('_')[1])-1], "\n")

print("Best 3 Few-shot Prompts:")
for name,_ in best_few:
    print(name, "\n", few_prompts[int(name.split('_')[1])-1], "\n")


Best 3 Zero-shot Prompts:
zero_4 
 بین دو جمله‌ی فارسی زیر چه رابطه‌ای برقرار است؟ فقط یکی از e/c/n بنویس.
جمله اول: {premise}
جمله دوم: {hypothesis}
پاسخ: 

zero_5 
 درک معنایی: آیا جمله دوم از جمله اول نتیجه می‌شود (e)، با آن در تضاد است (c)، یا مستقل است (n)؟
فرض: {premise}
فرضیه: {hypothesis}
پاسخ: 

zero_1 
 با توجه به جملات زیر، مشخص کن که جمله دوم با جمله اول چه رابطه‌ای دارد: 
گزینه‌ها: e=نتیجه‌گیری، c=تضاد، n=بی‌ربط.
فرض: {premise}
فرضیه: {hypothesis}
پاسخ فقط یکی از e، c یا n باشد. 

Best 3 Few-shot Prompts:
few_2 
 با توجه به مثال‌ها رابطه را مشخص کن:
    مثال۱: 
    فرض: رواج و پیدایش علم کلام، از مهمترین نتایج ورود اندیشه ها به جهان اسلام و تلاش و مساعی معتزلیان و مناظرات آنان با مخالفان بود.
    فرضیه: یکی از مهمترین نتایج ورود اندیشه‌ها به عالم اسلام و سعی و تلاش معتزلیان، رواج علم کلام است.
    → e

    مثال۲: 
    فرض: زادروز پروین اعتصامی (بیست و پنجم اسفندماه)، به عنوان «روز بزرگداشت پروین اعتصامی» نام‌گذاری شده‌است.
    فرضیه: ۲۵ اسفند در تقویم ایرانی، روز بزرگداشت 